In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections;

In [2]:
client = Socrata("data.sfgov.org", None)
# results = client.get("cuks-n6tp", limit = 2191368)
data = client.get("cuks-n6tp", limit = 3000000)

In [3]:
data_df = pd.DataFrame.from_records(data)

In [4]:
print(type(data_df))
print(data_df.shape)

<class 'pandas.core.frame.DataFrame'>
(2193138, 13)


In [5]:
print(data_df[:4])

                     address       category                     date  \
0      300 Block of MASON ST       BURGLARY  2017-11-23T00:00:00.000   
1   700 Block of MOULTRIE ST  LARCENY/THEFT  2017-12-29T00:00:00.000   
2  1200 Block of VALENCIA ST        ROBBERY  2017-09-21T00:00:00.000   
3    100 Block of ALEMANY BL       WARRANTS  2017-12-10T00:00:00.000   

  dayofweek                          descript incidntnum  \
0  Thursday          BURGLARY, UNLAWFUL ENTRY  170953428   
1    Friday      GRAND THEFT FROM LOCKED AUTO  186002910   
2  Thursday  SHOPLIFTING, FORCE AGAINST AGENT  170772240   
3    Sunday   ENROUTE TO OUTSIDE JURISDICTION  170998446   

                                            location  pddistrict  \
0  {u'type': u'Point', u'coordinates': [-122.4096...  TENDERLOIN   
1  {u'type': u'Point', u'coordinates': [-122.4161...   INGLESIDE   
2  {u'type': u'Point', u'coordinates': [-122.4208...     MISSION   
3  {u'type': u'Point', u'coordinates': [-122.4092...   INGLESIDE  

In [6]:
data_df.sort_values(by=['date'])

,address,category,date,dayofweek,descript,incidntnum,location,pddistrict,pdid,resolution,time,x,y
44112,100 Block of TAYLOR ST,OTHER OFFENSES,2003-01-01T00:00:00.000,Wednesday,ESCAPE FROM JAIL,030003983,"{u'type': u'Point', u'coordinates': [-122.4109...",TENDERLOIN,3000398326090,NONE,21:30,-122.410928353347,37.7836778741124
40564,3RD ST / CARGO WY,NON-CRIMINAL,2003-01-01T00:00:00.000,Wednesday,LOST PROPERTY,030000638,"{u'type': u'Point', u'coordinates': [-122.3871...",BAYVIEW,3000063871000,NONE,00:01,-122.387177992928,37.7461569686668
29162,0 Block of OCTAVIA ST,ASSAULT,2003-01-01T00:00:00.000,Wednesday,BATTERY,030192774,"{u'type': u'Point', u'coordinates': [-122.4238...",NORTHERN,3019277404134,NONE,16:00,-122.423898239301,37.7725026632238
40519,2100 Block of MISSION ST,DRUG/NARCOTIC,2003-01-01T00:00:00.000,Wednesday,SALE OF MARIJUANA,030003193,"{u'type': u'Point', u'coordinates': [-122.4194...",MISSION,3000319316040,"ARREST, BOOKED",19:52,-122.419470488515,37.7622701739914
22313,SUNNYDALE AV / SAWYER ST,VEHICLE THEFT,2003-01-01T00:00:00.000,Wednesday,"VEHICLE, RECOVERED, AUTO",030003814,"{u'type': u'Point', u'coordinates': [-122.4150...",INGLESIDE,3000381407041,NONE,21:00,-122.415080693167,37.7116558039525
29159,800 Block of FOLSOM ST,VANDALISM,2003-01-01T00:00:00.000,Wednesday,"MALICIOUS MISCHIEF, VANDALISM",030004492,"{u'type': u'Point', u'coordinates': [-122.4023...",SOUTHERN,3000449228150,NONE,03:00,-122.402343659494,37.7811666837563
22287,200 Block of CHATTANOOGA ST,LIQUOR LAWS,2003-01-01T00:00:00.000,Wednesday,POSSESSION OF ALCOHOL BY MINOR,030003773,"{u'type': u'Point', u'coordinates': [-122.4264...",MISSION,3000377317060,"ARREST, CITED",22:40,-122.426472976561,37.7525462209028
22245,300 Block of THE EMBARCADERO NORTH ST,MISSING PERSON,2003-01-01T00:00:00.000,Wednesday,FOUND PERSON,030000939,"{u'type': u'Point', u'coordinates': [-122.3963...",CENTRAL,3000093975000,LOCATED,00:59,-122.396374439116,37.7980532528058
26966,400 Block of BROADWAY ST,NON-CRIMINAL,2003-01-01T00:00:00.000,Wednesday,FOUND PROPERTY,030026026,"{u'type': u'Point', u'coordinates': [-122.4042...",CENTRAL,3002602672000,NONE,02:00,-122.404260458632,37.7981164818887
60447,NEWCOMB AV / PHELPS ST,WARRANTS,2003-01-01T00:00:00.000,Wednesday,WARRANT ARREST,030003911,"{u'type': u'Point', u'coordinates': [-122.3935...",BAYVIEW,3000391163010,"ARREST, BOOKED",23:33,-122.39352632602,37.7373529004232


In [7]:
print(data_df.shape)

(2193138, 13)


In [8]:
data_df = data_df[data_df.date < '2016-07-01T00:00:00.000' ]

In [9]:
print(data_df[:0])

Empty DataFrame
Columns: [address, category, date, dayofweek, descript, incidntnum, location, pddistrict, pdid, resolution, time, x, y]
Index: []


In [10]:
df = data_df.drop(['incidntnum','pdid'], axis =1)
# df = df.drop(df.columns[[0]], axis =1)

In [11]:
print(df.shape)

(1937307, 11)


In [12]:
df.sort_values(by=['date'])

,address,category,date,dayofweek,descript,location,pddistrict,resolution,time,x,y
34543,700 Block of LYON ST,NON-CRIMINAL,2003-01-01T00:00:00.000,Wednesday,"AIDED CASE, MENTAL DISTURBED","{u'type': u'Point', u'coordinates': [-122.4433...",PARK,PSYCHOPATHIC CASE,02:30,-122.443347430278,37.7776443519022
58199,100 Block of 4TH ST,ASSAULT,2003-01-01T00:00:00.000,Wednesday,BATTERY,"{u'type': u'Point', u'coordinates': [-122.4039...",SOUTHERN,"ARREST, CITED",21:00,-122.4039407276,37.7843007072344
24793,3100 Block of MISSION ST,NON-CRIMINAL,2003-01-01T00:00:00.000,Wednesday,"AIDED CASE, MENTAL DISTURBED","{u'type': u'Point', u'coordinates': [-122.4186...",INGLESIDE,NONE,09:46,-122.418609719102,37.7474453071495
484849,800 Block of POST ST,ASSAULT,2003-01-01T00:00:00.000,Wednesday,THREATS AGAINST LIFE,"{u'type': u'Point', u'coordinates': [-122.4158...",CENTRAL,NONE,01:00,-122.415821891164,37.7872932910877
31598,3600 Block of BALBOA ST,ASSAULT,2003-01-01T00:00:00.000,Wednesday,AGGRAVATED ASSAULT WITH BODILY FORCE,"{u'type': u'Point', u'coordinates': [-122.4978...",RICHMOND,NONE,17:05,-122.497866355514,37.7757454267928
47683,BARNEVELD AV / PALOU AV,LARCENY/THEFT,2003-01-01T00:00:00.000,Wednesday,GRAND THEFT FROM LOCKED AUTO,"{u'type': u'Point', u'coordinates': [-122.4036...",BAYVIEW,NONE,00:15,-122.40361089259,37.7414756041488
125335,0 Block of RAYMOND AV,FRAUD,2003-01-01T00:00:00.000,Wednesday,FRAUDULENT CREDIT APPLICATION,"{u'type': u'Point', u'coordinates': [-122.4043...",INGLESIDE,NONE,00:01,-122.404306598392,37.7121557428309
58169,2200 Block of 22ND AV,VANDALISM,2003-01-01T00:00:00.000,Wednesday,"MALICIOUS MISCHIEF, GRAFFITI","{u'type': u'Point', u'coordinates': [-122.4790...",TARAVAL,NONE,16:00,-122.479030398225,37.7456918936678
58165,400 Block of CASTRO ST,BURGLARY,2003-01-01T00:00:00.000,Wednesday,"BURGLARY OF STORE, UNLAWFUL ENTRY","{u'type': u'Point', u'coordinates': [-122.4351...",MISSION,NONE,02:30,-122.435150099812,37.7617597243598
37054,9TH AV / CLEMENT ST,VEHICLE THEFT,2003-01-01T00:00:00.000,Wednesday,"VEHICLE, RECOVERED, AUTO","{u'type': u'Point', u'coordinates': [-122.4676...",RICHMOND,UNFOUNDED,15:00,-122.467630230365,37.7828011737745


In [13]:
print(df.shape)

(1937307, 11)


In [14]:
# df.to_csv("test.csv", sep=',', encoding='utf-8')

In [15]:
print(df[:2])

                address      category                     date dayofweek  \
836  GEARY BL / 35TH AV  NON-CRIMINAL  2011-04-17T00:00:00.000    Sunday   
887    6TH AV / ANZA ST       ASSAULT  2007-03-02T00:00:00.000    Friday   

          descript                                           location  \
836  LOST PROPERTY  {u'type': u'Point', u'coordinates': [-122.4954...   
887        BATTERY  {u'type': u'Point', u'coordinates': [-122.4641...   

    pddistrict resolution   time                  x                 y  
836   RICHMOND       NONE  15:30  -122.495440324304  37.7795895470861  
887   RICHMOND       NONE  08:15  -122.464147081977  37.7790877048755  


In [16]:
print(type(df.groupby('pddistrict').apply(list)))
print(df.groupby('pddistrict').apply(list))

<class 'pandas.core.series.Series'>
pddistrict
BAYVIEW       [address, category, date, dayofweek, descript,...
CENTRAL       [address, category, date, dayofweek, descript,...
INGLESIDE     [address, category, date, dayofweek, descript,...
MISSION       [address, category, date, dayofweek, descript,...
NORTHERN      [address, category, date, dayofweek, descript,...
PARK          [address, category, date, dayofweek, descript,...
RICHMOND      [address, category, date, dayofweek, descript,...
SOUTHERN      [address, category, date, dayofweek, descript,...
TARAVAL       [address, category, date, dayofweek, descript,...
TENDERLOIN    [address, category, date, dayofweek, descript,...
dtype: object


In [89]:
series = df.groupby('pddistrict').apply(list)
dictionary = {}
for s in series.keys():
    print(str(s))
    mask = df['pddistrict'] == str(s)
    print(df[mask])
    dictionary[str(s)] = df[mask]

BAYVIEW
                              address         category  \
1139           700 Block of AMADOR ST   SUSPICIOUS OCC   
1192          1200 Block of DEHARO ST    LARCENY/THEFT   
1194             4400 Block of 3RD ST          ASSAULT   
1332               0 Block of HARE ST      WEAPON LAWS   
1333      900 Block of CONNECTICUT ST   OTHER OFFENSES   
1381        1500 Block of KIRKWOOD AV   OTHER OFFENSES   
1383             4700 Block of 3RD ST         WARRANTS   
1416               3RD ST / REVERE AV          ASSAULT   
1440        HARBOR RD / NORTHRIDGE RD   OTHER OFFENSES   
1509             2200 Block of 3RD ST      WEAPON LAWS   
1566       1500 Block of UNDERWOOD AV   OTHER OFFENSES   
1567                3RD ST / BURKE AV   OTHER OFFENSES   
1595        900 Block of JAMESTOWN AV  SECONDARY CODES   
1892             0 Block of MABREY CT            FRAUD   
10029           500 Block of TEXAS ST         BURGLARY   
10041           FELTON ST / GIRARD ST   OTHER OFFENSES   
10046 

CENTRAL
                                      address                category  \
1136                    600 Block of JONES ST           VEHICLE THEFT   
1273                      400 Block of BAY ST            NON-CRIMINAL   
1301                    3100 Block of POLK ST         STOLEN PROPERTY   
1304                BROADWAY ST / COLUMBUS AV          OTHER OFFENSES   
1387                CALIFORNIA ST / LARKIN ST                WARRANTS   
1410               VANNESS AV / NORTHPOINT ST                 ASSAULT   
1449                    500 Block of GEARY ST          MISSING PERSON   
1485                  1500 Block of POWELL ST       RECOVERED VEHICLE   
1507                1000 Block of STOCKTON ST           LARCENY/THEFT   
1510                  SACRAMENTO ST / HYDE ST                 ASSAULT   
1514                    200 Block of GEARY ST           LARCENY/THEFT   
1753               100 Block of SACRAMENTO ST           LARCENY/THEFT   
1754                    400 Block of MASON 

INGLESIDE
                                 address        category  \
1129               200 Block of ALPHA ST    NON-CRIMINAL   
1193                200 Block of 30TH ST   LARCENY/THEFT   
1352                300 Block of PARK ST    NON-CRIMINAL   
1415            200 Block of SCHWERIN ST    NON-CRIMINAL   
1481              500 Block of FRANCE AV  MISSING PERSON   
1486            500 Block of ARGONAUT AV    NON-CRIMINAL   
1508              200 Block of LOWELL ST       VANDALISM   
1561               DELTA ST / RAYMOND AV    NON-CRIMINAL   
1624            400 Block of MONTEREY BL  OTHER OFFENSES   
4058           1300 Block of BOSWORTH ST         ASSAULT   
7892             100 Block of VELASCO AV   VEHICLE THEFT   
10026           3300 Block of MISSION ST        WARRANTS   
10027             100 Block of AMAZON AV   LARCENY/THEFT   
10051              400 Block of PARIS ST        TRESPASS   
10076              GENEVA AV / SANTOS ST         ROBBERY   
10077              OTSEGO AV /

MISSION
                                address                category  \
1111           2700 Block of MISSION ST                 ASSAULT   
1279            900 Block of POTRERO AV                 RUNAWAY   
1305           2000 Block of MISSION ST          OTHER OFFENSES   
1306              17TH ST / GUERRERO ST           VEHICLE THEFT   
1324            400 Block of SANCHEZ ST           LARCENY/THEFT   
1327              2500 Block of 23RD ST          OTHER OFFENSES   
1357            2000 Block of BRYANT ST           LARCENY/THEFT   
1361            2000 Block of MARKET ST                BURGLARY   
1385           2100 Block of MISSION ST                 ROBBERY   
1388            600 Block of CLIPPER ST            NON-CRIMINAL   
1447           2200 Block of MISSION ST           LARCENY/THEFT   
1451               19TH ST / DIAMOND ST                WARRANTS   
1453              3100 Block of 24TH ST          SUSPICIOUS OCC   
1479           600 Block of VALENCIA ST            NON

NORTHERN
                             address                category  \
1138            CLAY ST / VANNESS AV            PROSTITUTION   
1191         SUTTER ST / FILLMORE ST          SUSPICIOUS OCC   
1328             POLK ST / SUTTER ST           LARCENY/THEFT   
1334         LARKIN ST / OFARRELL ST           LARCENY/THEFT   
1354           300 Block of HAYES ST            NON-CRIMINAL   
1355          SUTTER ST / VANNESS AV           LARCENY/THEFT   
1356           1800 Block of POST ST                WARRANTS   
1360         800 Block of STEINER ST           LARCENY/THEFT   
1413         1200 Block of SUTTER ST         SECONDARY CODES   
1450             GEARY BL / GOUGH ST           LARCENY/THEFT   
1483        FILLMORE ST / VALLEJO ST            NON-CRIMINAL   
1484          POLK ST / FRANCISCO ST          OTHER OFFENSES   
1487       1700 Block of BUCHANAN ST                BURGLARY   
1512          100 Block of MARINA BL            NON-CRIMINAL   
1556       GOUGH ST / GOLDEN GA

PARK
                                address        category  \
1128            1400 Block of HAIGHT ST    NON-CRIMINAL   
1300             2200 Block of HAYES ST    NON-CRIMINAL   
1379              GEARY BL / MASONIC AV       VANDALISM   
1386           GROVE ST / DIVISADERO ST    NON-CRIMINAL   
1414            1400 Block of HAIGHT ST  OTHER OFFENSES   
1448            1600 Block of HAIGHT ST    NON-CRIMINAL   
1511         400 Block of DIVISADERO ST        WARRANTS   
1560          500 Block of FREDERICK ST   LARCENY/THEFT   
1622               1200 Block of 3RD AV  OTHER OFFENSES   
10052               HAIGHT ST / COLE ST   LARCENY/THEFT   
10057            2100 Block of GEARY BL         ASSAULT   
10080               BAKER ST / GEARY BL    NON-CRIMINAL   
10086               HAIGHT ST / LYON ST        WARRANTS   
10104            1200 Block of SCOTT ST         ASSAULT   
10111           1600 Block of HAIGHT ST    NON-CRIMINAL   
10113              1800 Block of OAK ST         ASS

RICHMOND
                                    address                    category  \
836                      GEARY BL / 35TH AV                NON-CRIMINAL   
887                        6TH AV / ANZA ST                     ASSAULT   
1131           500 Block of JOHNFKENNEDY DR               DRUG/NARCOTIC   
1331                   GEARY BL / SPRUCE ST              OTHER OFFENSES   
1358                     GEARY BL / 14TH AV              OTHER OFFENSES   
1389                     0 Block of 21ST AV               LARCENY/THEFT   
1452                      8TH AV / GEARY BL              OTHER OFFENSES   
1482           500 Block of JOHNFKENNEDY DR               LARCENY/THEFT   
1558                 GEARY BL / ARGUELLO BL              OTHER OFFENSES   
1596                 800 Block of EUCLID AV              MISSING PERSON   
9220                500 Block of CLEMENT ST                NON-CRIMINAL   
10069                  700 Block of 23RD AV              MISSING PERSON   
10082    25TH AV

SOUTHERN
                                      address                     category  \
1130                      300 Block of 6TH ST               OTHER OFFENSES   
1132              0 Block of UNITEDNATIONS PZ                 NON-CRIMINAL   
1134                   800 Block of BRYANT ST                LARCENY/THEFT   
1135                   800 Block of BRYANT ST                      ASSAULT   
1137                   800 Block of BRYANT ST                DRUG/NARCOTIC   
1274                   800 Block of BRYANT ST                LARCENY/THEFT   
1275                    400 Block of SPEAR ST                LARCENY/THEFT   
1277               300 Block of CLEMENTINA ST                     BURGLARY   
1303                 1100 Block of MISSION ST               MISSING PERSON   
1325                       0 Block of MOSS ST               MISSING PERSON   
1326                   500 Block of HOWARD ST               SUSPICIOUS OCC   
1329                      4TH ST / MISSION ST          

TARAVAL
                                 address                category  \
1195                PACHECO ST / 37TH AV          OTHER OFFENSES   
1278               1200 Block of 34TH AV            NON-CRIMINAL   
1384               2000 Block of 37TH AV          SUSPICIOUS OCC   
1408               3200 Block of 20TH AV           LARCENY/THEFT   
1409               0 Block of MIRAMAR AV          MISSING PERSON   
1441                0 Block of CORONA ST            NON-CRIMINAL   
1444            3300 Block of TARAVAL ST            NON-CRIMINAL   
1446                  46TH AV / JUDAH ST                WARRANTS   
4455                18TH AV / TARAVAL ST          OTHER OFFENSES   
10028              3200 Block of 20TH AV            NON-CRIMINAL   
10033               1200 Block of 9TH AV          OTHER OFFENSES   
10048            200 Block of WINSTON DR          OTHER OFFENSES   
10081          SAGAMORE ST / SAN JOSE AV            NON-CRIMINAL   
10083              2400 Block of 42ND AV

TENDERLOIN
                              address            category  \
1133              0 Block of MASON ST        NON-CRIMINAL   
1276            300 Block of ELLIS ST      OTHER OFFENSES   
1302               0 Block of TURK ST      OTHER OFFENSES   
1382           0 Block of STOCKTON ST       LARCENY/THEFT   
1411                TURK ST / HYDE ST       LARCENY/THEFT   
1412             100 Block of EDDY ST             SUICIDE   
6999         EDDY ST / LEAVENWORTH ST             ASSAULT   
10036            0 Block of POWELL ST               FRAUD   
10043           500 Block of ELLIS ST            WARRANTS   
10068           200 Block of JONES ST               FRAUD   
10107            LARKIN ST / GROVE ST       DRUG/NARCOTIC   
10117          200 Block of POWELL ST            BURGLARY   
10120          200 Block of POWELL ST       DRUG/NARCOTIC   
10136             EDDY ST / TAYLOR ST       DRUG/NARCOTIC   
10142             0 Block of ELLIS ST               FRAUD   
10145        

In [92]:
print(type(dictionary['RICHMOND']))
print(dictionary.keys())

<class 'pandas.core.frame.DataFrame'>
['CENTRAL', 'NORTHERN', 'INGLESIDE', 'PARK', 'MISSION', 'TENDERLOIN', 'RICHMOND', 'TARAVAL', 'BAYVIEW', 'SOUTHERN']


10


In [ ]:
mask = results_df['pddistrict'] == 'RICHMOND'
df_RICHMOND = results_df[mask]
df_RICHMOND.to_csv('RICHMOND_data', index=False, header=True, columns= ['address', 'category', 'date', 'dayofweek', 'descript', 'incidntnum',
      'location', 'pddistrict', 'pdid', 'resolution', 'time', 'x', 'y'])